In [80]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import Dict

model = Ollama(model="gemma")

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field





# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# # And a query intended to prompt a language model to populate the data structure.
# prompt_and_model = prompt | model
# output = prompt_and_model.invoke({"query": "tell me a joke."})
# parser.invoke(output)

In [86]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Dict

# Define the model
model = ChatOllama(model = 'gemma')

# Defining the Pydantic class
class SmartReplies(BaseModel):
    reply_1: str = Field(description = "Smart Reply 1")
    reply_2: str = Field(description = "Smart Reply 2")
    reply_3: str = Field(description =" Smart Reply 3")

        
parser = PydanticOutputParser(pydantic_object=SmartReplies)

In [97]:
# Define template

from langchain_core.prompts.chat import ChatMessage
from langchain.memory import ChatMessageHistory


template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to the responder. Your role is to suggest exactly {num_reply} distinct responses for the responder, each with fewer than {num_words} words, based on the conversation history with the responder and the human. Please be truthful, succinct and clear. The history is attached here \n{chat_history}\n. And follow the formatting instruction: \n{format_instruction}",),
    ("human",  "{message}")
])

model = ChatOllama(model = 'gemma', temperature = 0.6)

chain = template | model | parser

def parse_history(chat_history: ChatMessageHistory):
    result = []
    for message in chat_history.messages:
        if hasattr(message, 'type'):
            if message.type == 'human':
                result.append(f"human: {message.content}")
            elif message.type == 'ai':
                result.append(f"ai: {message.content}")
            elif message.type == 'chat':
                 result.append(f"{message.role}: {message.content}")
    return '\n'.join(result)

In [93]:
# Add memory retrieval

def get_replies(message: str, chat_history: ChatMessageHistory, save_to_history: bool = False) -> SmartReplies:
    # if not chat_history:
    #     chat_history = ChatMessageHistory()
    response = chain.invoke({"message": message, 
                  "num_reply": 3,
                  "num_words": 10,
                  "format_instruction": parser.get_format_instructions(),
                  "chat_history": parse_history(chat_history)})
    if save_to_history:
        chat_history.add_user_message(message)
    return response

def reply(message: str, chat_history: ChatMessageHistory):
    chat_history.add_message(ChatMessage(role = "responder", content = message))

## Test run

In [102]:
# Test run

memory_1 = ChatMessageHistory()

get_replies(message="Hi. Can I see what time it is?", chat_history=memory_1, save_to_history=True)

SmartReplies(reply_1='Current time is 8:15 PM.', reply_2='The time is displayed at the top of the screen.', reply_3='Check the system tray for the time.')

In [103]:
reply(message="It's 9AM", chat_history=memory_1)

In [104]:
get_replies(message="So is it nighttime or morning time?", chat_history=memory_1, save_to_history=True)

SmartReplies(reply_1='Morning!', reply_2='The day has just begun!', reply_3='Time for a productive day!')

In [105]:
reply(message="Morning time", chat_history=memory_1)

In [106]:
get_replies(message="What is current time in 24hr format?", chat_history=memory_1)

SmartReplies(reply_1="It's 9AM in 24hr format.", reply_2='The current time is 09:00.', reply_3='Current time is 9:00 AM.')

In [7]:

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), human_message_template]
)